In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import colorsys
from tqdm import tqdm

In [ ]:
# Read csv file into pandas dataframe
# original file cpu4.csv is from https://www.kaggle.com/code/caesarlupum/ar-labeled-anomaly-detection-in-time-series/data.
filename = 'cpu4_full'
df = pd.read_csv('csv_files/' + filename + '.csv', sep = ',')
df['date']=(pd.to_datetime(df['date'],unit='s')) # assume unix timestamp from input always in sec

In [ ]:
df.head

In [ ]:
# Helper function make color (hex string/color string) lighter by multiplying luminosity and luminance_scale
def lighten(color, luminance_scale):
    try:
        c = colors.cnames[color]
    except:
        c = color
    h, l, s = colorsys.rgb_to_hls(*colors.to_rgb(c))
    return colorsys.hls_to_rgb(h, 1 - luminance_scale * (1 - l), s)

In [ ]:
output_folder = 'plots'
fig = plt.figure(figsize=(35,10))
ax = fig.add_subplot(111)

# plot original timeseries/prophet expected timeseries/egads expected timeseries
ax.plot(df.date, df.original, label='Original Value', color='tomato')
ax.plot(df.date, df.prophetExpected, label='Prophet Predicted', color=lighten('b',0.15))
       
# plot anomalies
both_anomaly_flag = False
prophet_anomaly_flag = False
miss_anomaly_flag = False
prophet_true_pos_cnt = 0
prophet_true_neg_cnt = 0
prophet_false_pos_cnt = 0
for idx, row in tqdm(df.iterrows()):
    if row['prophetAnomaly'] == 'yes' and row['label'] == 1:
        prophet_true_pos_cnt += 1
        if not both_anomaly_flag:
            ax.scatter(row['date'], row['original'], color='red', label="True anomaly identified by Prophet")
            both_anomaly_flag = True
        else:
            ax.scatter(row['date'], row['original'], color='red')
    elif row['label'] == 1:
        prophet_true_neg_cnt += 1
        if not miss_anomaly_flag:
            ax.scatter(row['date'], row['original'], color='green', label="True anomaly not identified by Prophet")       
            miss_anomaly_flag = True
        else:
            ax.scatter(row['date'], row['original'], color='green')              
    elif row['prophetAnomaly'] == 'yes':
        prophet_false_pos_cnt += 1
        if not prophet_anomaly_flag:
            ax.scatter(row['date'], row['original'], color='blue', label="False positives by Prophet")
            prophet_anomaly_flag = True
        else:
            ax.scatter(row['date'], row['original'], color='blue')
ax.legend(loc = 'best')
plt.title("Visualize Prophet anomalies vs true anomalies")
plt.savefig(output_folder + '/' + filename + '.png')

In [ ]:
print("prophet_true_pos_cnt", prophet_true_pos_cnt)
print("prophet_true_neg_cnt", prophet_true_neg_cnt)
print("prophet_false_pos_cnt", prophet_false_pos_cnt)